In [ ]:
import pandas as pd
import uuid
import json
import os

# Dossier contenant les fichiers CSV
folder_path = "consolide"  

# Fonction pour créer un format JSON à partir des données CSV
def convert_csv_to_json(file_path):
    try:
        # Charger le fichier CSV
       # df = pd.read_csv(file_path, sep=';')
        df = pd.read_csv(file_path, sep=';', encoding="utf-8")

        # Reconstituer le texte complet en s'assurant que tous les tokens sont des chaînes
        tokens = df["Token"].fillna("").astype(str).tolist()  # Remplacer les NaN par une chaîne vide et convertir en str
       # full_text = " ".join(tokens)
        #full_text = " ".join(df["Token"].dropna().astype(str)).strip()
        # Reconstituer le texte complet sans espaces superflus ni sauts de ligne
        full_text = " ".join(df["Token"].dropna().astype(str)).replace("\n", "").strip()

        # Statistiques pour le fichier
        num_tokens = len(tokens)
        num_annotations = 0
        num_quaero_annotations = 0

        # Préparer les annotations
        annotations = []
        for i, row in df.iterrows():
            token = str(row["Token"])  # Convertir en chaîne de caractères
            label = row["Label_Maj"]
            quaero_label = row["Quaero_Maj"] if pd.notna(row["Quaero_Maj"]) else None

            # Si l'annotation est présente
            if pd.notna(label):
                # Calculer la position du token dans le texte complet
                start = full_text.find(token)
                end = start + len(token)

                # Ajouter l'annotation au format requis
                annotation = {
                    "id": str(uuid.uuid4()),
                    "tag_id": str(uuid.uuid4()),  # Tag ID unique
                    "start": start,
                    "end": end,
                    "example_id": str(uuid.uuid4()),
                    "tag_name": label,
                    "value": token,
                    "correct": None,
                    "human_annotations": [],
                    "model_annotations": []
                }

                # Si c'est une entité de type "LOC", vérifier si Quaero est aussi disponible
                if label == "LOC" and quaero_label:
                    annotation["tag_name"] = f"LOC, {quaero_label}"  # Ajouter Quaero à l'annotation
                    num_quaero_annotations += 1

                annotations.append(annotation)
                num_annotations += 1

        # Préparer l'exemple
        example = {
            "id": str(uuid.uuid4()),
            "content": full_text,
            "metadata": {},
            "annotations": annotations
        }

        # Suivi des statistiques pour l'affichage
        print(f"📂 Fichier : {file_path}")
        print(f"   🔹 Nombre de tokens : {num_tokens}")
        print(f"   🔹 Nombre d'annotations : {num_annotations}")
        print(f"   🔹 Nombre d'annotations Quaero : {num_quaero_annotations}")
        
        # Retourner les données au format JSON
        return {"examples": [example]}

    except Exception as e:
        print(f"❌ Erreur avec {file_path} : {e}")
        return None

# Liste pour accumuler les données JSON de tous les fichiers
all_data = {"examples": []}

# Parcourir tous les fichiers du dossier
for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):  # Vérifie que c'est un fichier CSV
        file_path = os.path.join(folder_path, file_name)
        
        # Convertir le fichier CSV en format JSON
        json_data = convert_csv_to_json(file_path)
        
        if json_data:
            # Ajouter les données au fichier JSON principal
            all_data["examples"].extend(json_data["examples"])

# Sauvegarder tous les fichiers JSON dans un fichier de sortie unique
output_file_path = "combined_data_output.json"
with open(output_file_path, 'w', encoding="utf-8") as json_file:
    json.dump(all_data, json_file, indent=4, ensure_ascii=False)

print(f"✅ Tous les fichiers ont été combinés et sauvegardés sous '{output_file_path}'")